In [1]:
import jax.numpy as np
import numpy as onp
from functools import partial
from jax import vmap
from jax.lax import scan
from jax.lax import cond
from jax import random
from jax import jit
from jax import jacrev
from jax.lax import stop_gradient
import matplotlib.pylab as plt
import matplotlib as mpl
from sklearn import manifold, datasets
import seaborn as sns
import jax

from jax.config import config
config.update("jax_debug_nans", True)

import jax.numpy as np
from utils import MidpointNormalize, load_data
from jax import random, flatten_util, vjp, jvp, custom_vjp, jacfwd, jacrev, vmap, grad
from IFD_tsne import tsne_fwd
import jax
import matplotlib.pylab as plt
import seaborn as sns

In [2]:
X, y = load_data(50)
key = random.PRNGKey(41)
#X = onp.array(random.normal(key, shape=(50, 50)))
y_guess = random.normal(key, shape=(X.shape[0], 2))
#Y_star = TSNE(n_components=2, learning_rate=200, init=onp.array(y_guess), perplexity=30).fit_transform(X)
Y_star = tsne_fwd(X, y_guess)

X_flat, X_unflattener = flatten_util.ravel_pytree(X)   # row-wise
Y_flat, Y_unflattener = flatten_util.ravel_pytree(Y_star) 

I0000 00:00:1717683519.351337       1 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.


===> Finding 49 nearest neighbors using Annoy approximate search using euclidean distance...
   --> Time elapsed: 0.01 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.00 seconds
===> Running optimization with exaggeration=12.00, lr=200.00 for 250 iterations...
Iteration   50, KL divergence 1.0229, 50 iterations in 0.2916 sec
Iteration  100, KL divergence 0.9836, 50 iterations in 0.2799 sec
Iteration  150, KL divergence 0.9226, 50 iterations in 0.2889 sec
Iteration  200, KL divergence 0.8615, 50 iterations in 0.2673 sec
Iteration  250, KL divergence 0.8316, 50 iterations in 0.2717 sec
   --> Time elapsed: 1.40 seconds
===> Running optimization with exaggeration=1.00, lr=200.00 for 750 iterations...
Iteration   50, KL divergence 0.1430, 50 iterations in 0.2694 sec
Iteration  100, KL divergence 0.1427, 50 iterations in 0.2688 sec
Iteration  150, KL divergence 0.1427, 50 iterations in 0.2669 sec
Iteration  200, KL divergence 0.1427, 50 iterations in 0.2702 sec
Iteration 

In [3]:
# Pairwise squared Euclidean distance
def get_dists(Z):
    """
    Compute all pairwise distances of each data points z_i in Z.
    
    Params:
        Z: matrix with n rows. Each row is the z_i
        
    Return:
        All pairwise distances of each data points z_i in Z
    """

    diff = Z[:, :, None] - Z[:, :, None].T
    return (np.square(diff)).sum(1)

In [4]:
def logSoftmax(x):
    """Compute softmax for vector x."""
    max_x = np.max(x)
    exp_x = np.exp(x - max_x)
    sum_exp_x = np.sum(exp_x)
    log_sum_exp_x = np.log(sum_exp_x)
    max_plus_log_sum_exp_x = max_x + log_sum_exp_x
    log_probs = x - max_plus_log_sum_exp_x

    # Recover probs
    exp_log_probs = np.exp(log_probs)
    sum_log_probs = np.sum(exp_log_probs)
    probs = exp_log_probs / sum_log_probs
    return probs

def Hbeta(D: np.ndarray, beta=1.0):
    """
    Compute the log2(perplexity)=Entropy and the P-row (P_i) for a specific value of the
        precision=1/(sigma**2) (beta) of a Gaussian distribution. D: vector of squared Euclidean distances (without i)
    :param D: vector of length d, squared Euclidean distances to all other datapoints (except itself)
    :param beta: precision = beta = 1/sigma**2
    :return: H: log2(Entropy), P: computed probabilites
    """
    # TODO: exchange by softmax as described by https://nlml.github.io/in-raw-numpy/in-raw-numpy-t-sne/
    P = np.exp(-D * beta)     # numerator of p j|i
    sumP = np.sum(P, axis=None)    # denominator of p j|i --> normalization factor
    new_P = logSoftmax(-D * beta)
    sumP += 1e-8
    H = np.log(sumP) + beta * np.sum(D * P) / sumP
    return H, new_P

def HdiffGreaterTrue(*betas):
    beta, betamax = betas
    return beta*2

def HdiffGreaterFalse(*betas):
    beta, betamax = betas
    return (beta+betamax)/2

def HdiffSmallerTrue(*betas):
    beta, betamin = betas
    return beta/2

def HdiffSmallerFalse(*betas):
    beta, betamin = betas
    return (beta+betamin)/2

def HdiffGreater(*betas):
    beta, betamin, betamax = betas
    betamin = beta
    beta = cond((np.logical_or(betamax == np.inf, betamax == -np.inf)), HdiffGreaterTrue, HdiffGreaterFalse, *(beta, betamax))
    return beta, betamin, betamax

def HdiffSmaller(*betas):
    beta, betamin, betamax = betas
    betamax = beta
    beta = cond(np.logical_or(betamin == np.inf, betamin == -np.inf), HdiffSmallerTrue, HdiffSmallerFalse, *(beta, betamin))
    return beta, betamin, betamax

def HdiffGreaterTolerance(*betas):
    beta, betamin, betamax, Hdiff = betas
    beta, betamin, betamax = cond(Hdiff > 0, HdiffGreater, HdiffSmaller, *(beta, betamin, betamax))
    return beta, betamin, betamax, Hdiff

def binarySearch(res, el, Di, logU):
    print('Entered binary search function')
    Hdiff, thisP, beta, betamin, betamax = res
    beta, betamin, betamax, Hdiff = cond(np.abs(Hdiff) < 1e-5, lambda a, b, c, d: (a, b, c, d), HdiffGreaterTolerance, *(beta, betamin, betamax, Hdiff))
    (H, thisP) = Hbeta(Di, beta)
    Hdiff = H - logU
    return (Hdiff, thisP, beta, betamin, betamax), el

def x2beta_inner(Di: np.ndarray, iterator, beta, betamin, betamax, perplexity=30, tol=1e-5):
    """
    binary search for precision for Pi such that it matches the perplexity defined by the user
    :param Di: vector of length d-1, squared Euclidean distances to all other datapoints (except itself)
    :param beta: precision = beta = 1/sigma**2
    :return: final probabilites p j|i
    """
    # Compute the Gaussian kernel and entropy for the current precision
    logU = np.log(perplexity)
    H, thisP = Hbeta(Di, beta)
    Hdiff = H - logU

    print('Starting binary search')
    binarySearch_func = partial(binarySearch, Di=Di, logU=logU)

    # Note: the following binary Search for suitable precisions (betas) will be repeated 50 times and does not include the threshold value
    (Hdiff, thisP, beta, betamin, betamax), el = scan(binarySearch_func, init=(Hdiff, thisP, beta, betamin, betamax), xs=None, length=1000)    # Set the final row of P
    #thisP = np.insert(thisP, iterator, 0)
    return beta

def x2beta(D: np.ndarray, tol=1e-5, perplexity=30.0):
    """
        Performs a binary search to get P-values (high-dim space) in such a way that each
        conditional Gaussian has the same perplexity.
    """
    # Initialize some variables
    n = D.shape[0]
    beta = np.ones(n)      # precisions (1/sigma**2)
    betamin = np.full(n, -np.inf)
    betamax = np.full(n, np.inf)
    betas_final = vmap(partial(x2beta_inner, perplexity=perplexity, tol=tol))(D, np.arange(n), beta=beta, betamin=betamin, betamax=betamax)
    return betas_final

def x2distance(X):
    print("Computing pairwise distances...")
    sum_X = np.sum(np.square(X), 1)
    (n, d) = X.shape
    D = np.add(np.add(-2 * np.dot(X, X.T), sum_X).T, sum_X)
    D = np.reshape(np.delete(D, np.array([i for i in range(0, D.shape[0]**2, (D.shape[0]+1))])), (n , n - 1 ))
    return D

def distance2p(D, betas):
    _, P_final = vmap(Hbeta, in_axes=0)(D, betas)
    print('P_final', P_final, P_final.shape)
    P_final = vmap(partial(np.insert, values=0))(P_final, np.arange(P_final.shape[0]))
    return P_final

def y2q(Y: np.ndarray):
    # Compute pairwise affinities
    sum_Y = np.sum(np.square(Y), 1)
    num = -2. * np.dot(Y, Y.T)  # numerator
    num = 1. / (1. + np.add(np.add(num, sum_Y).T, sum_Y))
    num = num.at[np.diag_indices_from(num)].set(0.)     # numerator
    Q = num / np.sum(num)
    Q = np.maximum(Q, 1e-12)
    return Q, num

def KL_divergence(X_flat, Y_flat, X_unflattener, Y_unflattener):
    """
    (R^nxp x R^nxp)--> R
    """
    X = X_unflattener(X_flat)
    Y = Y_unflattener(Y_flat)
    learning_rate, perplexity = (200, 30.0)
    D = x2distance(X)
    print('D', D.shape)
    betas = x2beta(jax.lax.stop_gradient(D), tol=1e-5, perplexity=perplexity)
    print('betas', betas.shape, betas)
    P = distance2p(D, betas)
    P = (P + np.transpose(P))
    P = P / np.sum(P)      # Why don't we devide by 2N as described everywhere?
    P = np.maximum(P, 1e-12)
    print('P', P, P.shape)
    Q, _ = y2q(Y)
    #print('Q', Q)
    return np.sum(P * (np.log(P+1e-10) - np.log(Q+1e-10)))

In [5]:
KL_divergence(X_flat, Y_flat, X_unflattener, Y_unflattener)

Computing pairwise distances...
D (50, 49)
Starting binary search
Entered binary search function
betas (50,) [0.03145981 0.03915882 0.05973053 0.04076576 0.06607628 0.04049015
 0.04353237 0.06218815 0.06668854 0.03736973 0.06702614 0.06758118
 0.0565815  0.05062008 0.0314188  0.02456474 0.03342628 0.03619146
 0.06592751 0.046278   0.07269287 0.02403736 0.04329967 0.03188038
 0.05447006 0.0561924  0.06148338 0.04558754 0.06275368 0.04301071
 0.05408669 0.03538179 0.04439449 0.03304195 0.0520668  0.03093863
 0.04922009 0.04143238 0.06144142 0.02959728 0.05911636 0.03255177
 0.08343315 0.03123474 0.06093407 0.03836107 0.0480032  0.04756737
 0.05224228 0.03555202]
P_final [[0.02984342 0.031716   0.01633002 ... 0.01053061 0.0070284  0.05538167]
 [0.02680009 0.01891257 0.0075951  ... 0.01321529 0.02116661 0.11250944]
 [0.06214235 0.03253066 0.00527259 ... 0.00464345 0.01524969 0.05626182]
 ...
 [0.01291137 0.02642421 0.00866591 ... 0.02217351 0.01916511 0.01210127]
 [0.00461468 0.0371759  0.

Array(0.1426619, dtype=float32)

In [6]:
J_X = jacfwd(KL_divergence, argnums=0)(X_flat, Y_flat, X_unflattener, Y_unflattener)
print('J', J_X)

Computing pairwise distances...
D (50, 49)
Starting binary search
Entered binary search function
betas (50,) [0.03145981 0.03915882 0.05973053 0.04076576 0.06607628 0.04049015
 0.04353237 0.06218815 0.06668854 0.03736973 0.06702614 0.06758118
 0.0565815  0.05062008 0.0314188  0.02456474 0.03342628 0.03619146
 0.06592751 0.046278   0.07269287 0.02403736 0.04329967 0.03188038
 0.05447006 0.0561924  0.06148338 0.04558754 0.06275368 0.04301071
 0.05408669 0.03538179 0.04439449 0.03304195 0.0520668  0.03093863
 0.04922009 0.04143238 0.06144142 0.02959728 0.05911636 0.03255177
 0.08343315 0.03123474 0.06093407 0.03836107 0.0480032  0.04756737
 0.05224228 0.03555202]
P_final Traced<ConcreteArray([[0.02984342 0.031716   0.01633002 ... 0.01053061 0.0070284  0.05538167]
 [0.02680009 0.01891257 0.0075951  ... 0.01321529 0.02116661 0.11250944]
 [0.06214235 0.03253066 0.00527259 ... 0.00464345 0.01524969 0.05626182]
 ...
 [0.01291137 0.02642421 0.00866591 ... 0.02217351 0.01916511 0.01210127]
 [0.0

In [7]:
from simplified_tsne_jax import * 
def KL_divergence_sim(X_flat, Y_flat, X_unflattener, Y_unflattener):
    """
    (R^nxp x R^nxp)--> R
    """
    X = X_unflattener(X_flat)
    Y = Y_unflattener(Y_flat)
    learning_rate, perplexity = (200, 30.0)
    P = x2p(X, tol=1e-5, perplexity=perplexity)
    P = (P + np.transpose(P))
    P = P / np.sum(P)      # Why don't we devide by 2N as described everywhere?
    P = np.maximum(P, 1e-12)
    print('P', P)
    Q, _ = y2q(Y)
    #print('Q', Q)
    return np.sum(P * (np.log(P+1e-10) - np.log(Q+1e-10)))

In [8]:
KL_divergence_sim(X_flat, Y_flat, X_unflattener, Y_unflattener)

Computing pairwise distances...
Starting binary search
Entered binary search function
beta Traced<ShapedArray(float32[])>with<BatchTrace(level=1/0)> with
  val = Array([0.03145981, 0.03915882, 0.05973053, 0.04076576, 0.06607628,
       0.04049015, 0.04353237, 0.06218815, 0.06668854, 0.03736973,
       0.06702614, 0.06758118, 0.0565815 , 0.05062008, 0.0314188 ,
       0.02456474, 0.03342628, 0.03619146, 0.06592751, 0.046278  ,
       0.07269287, 0.02403736, 0.04329967, 0.03188038, 0.05447006,
       0.0561924 , 0.06148338, 0.04558754, 0.06275368, 0.04301071,
       0.05408669, 0.03538179, 0.04439449, 0.03304195, 0.0520668 ,
       0.03093863, 0.04922009, 0.04143238, 0.06144142, 0.02959728,
       0.05911636, 0.03255177, 0.08343315, 0.03123474, 0.06093407,
       0.03836107, 0.0480032 , 0.04756737, 0.05224228, 0.03555202],      dtype=float32)
  batch_dim = 0
P [[1.0000000e-12 5.6643499e-04 9.3858340e-04 ... 2.3441980e-04
  1.1643085e-04 1.1474519e-03]
 [5.6643499e-04 1.0000000e-12 5.1443

Array(0.14266193, dtype=float32)

In [9]:
J_X = jacfwd(KL_divergence_sim, argnums=0)(X_flat, Y_flat, X_unflattener, Y_unflattener)
print('J', J_X)

Computing pairwise distances...
Starting binary search
Entered binary search function
beta Traced<ShapedArray(float32[])>with<BatchTrace(level=3/0)> with
  val = Array([0.03145981, 0.03915882, 0.05973053, 0.04076576, 0.06607628,
       0.04049015, 0.04353237, 0.06218815, 0.06668854, 0.03736973,
       0.06702614, 0.06758118, 0.0565815 , 0.05062008, 0.0314188 ,
       0.02456474, 0.03342628, 0.03619146, 0.06592751, 0.046278  ,
       0.07269287, 0.02403736, 0.04329967, 0.03188038, 0.05447006,
       0.0561924 , 0.06148338, 0.04558754, 0.06275368, 0.04301071,
       0.05408669, 0.03538179, 0.04439449, 0.03304195, 0.0520668 ,
       0.03093863, 0.04922009, 0.04143238, 0.06144142, 0.02959728,
       0.05911636, 0.03255177, 0.08343315, 0.03123474, 0.06093407,
       0.03836107, 0.0480032 , 0.04756737, 0.05224228, 0.03555202],      dtype=float32)
  batch_dim = 0
P Traced<ConcreteArray([[1.0000000e-12 5.6643499e-04 9.3858340e-04 ... 2.3441980e-04
  1.1643085e-04 1.1474519e-03]
 [5.6643499e-04